In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
from dotenv import load_dotenv
import requests
from urllib.parse import urljoin
from datetime import datetime, timedelta
import io
from io import BytesIO
import base64
from email.mime.image import MIMEImage
import openpyxl
import time
import unicodedata2 as unicodedata

In [2]:
#----------------------------------------------------------------------------------------------- 
# TODO: Carregar as variáveis de ambiente
load_dotenv(override=True)


api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

# TODO: Configurações do e-mail
enviar_para = os.getenv('DESTINATARIO')
print(f"Valor original do DESTINATARIO: {enviar_para}")

if enviar_para:
    enviar_para = [email.strip() for email in enviar_para.split(',')]
else:
    enviar_para = []

enviar_para_dois = os.getenv('DESTINATARIO2')
print(f"Valor original do DESTINATARIO: {enviar_para_dois}")

if enviar_para_dois:
    enviar_para_dois = [email.strip() for email in enviar_para_dois.split(',')]
else:
    enviar_para_dois = []

username_email = os.getenv('EMAIL_USERNAME')
password_email = os.getenv('EMAIL_PASSWORD')
server_email = os.getenv('EMAIL_SERVER')
port_email = int(os.getenv('EMAIL_PORT'))


# TODO: variáveis de período
mes_atual = datetime.now().month - 0 
ano_atual = datetime.now().year
ano_anterior = datetime.now().year - 1
proximas_duas_semanas = (datetime.now()+timedelta(days=15)).strftime('%Y-%m-%d')
duas_ultimas_semanas = (datetime.now()-timedelta(days=15))
ultima_semana = (datetime.now() - timedelta(days=7))
proxima_semana = (datetime.now()+timedelta(days=7))
amanha = (datetime.now()+timedelta(days=0))
ontem = (datetime.now()-timedelta(days=1))

# TODO: Configuração CSS
css_hover = """
<style>
    tr:hover {
        background-color: #EC0E73 !important;
        color: #FFFFFF;
    }
    div {
        margin: 40px 0px 40px 0 px;
        overflow-x:auto;
    }
    h1 {
  font-size: 30px;
    }

    h2 {
    font-size: 20px;
    }

    p {
    font-size: 16px;
    }
    
    th {
        font-size: 16px;
        font-weight: bold;
        background-color: #041266;
        color: #FFFFFF;
        white-space: nowrap;
    }
    td {
        font-size: 14px 
    }
</style>

"""     
## --------------------------EXTRAIR INFORMAÇÕES DE BIBLIOTECAS ---------------------------------
def extrair_ultima_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[-1]


def extrair_segunda_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[1]
        
def extrair_terceira_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[2] if len(values_list) > 2 else None

        
def extrair_apelido_protocolo(x):
    if x is None:
        return None
    elif 'apelido_protocolo' in x:
        return x['apelido_protocolo']
    else:
        return None
    
    
def extrair_campo(x, *chaves):
# """
# Extrai um campo aninhado de um dicionário dado um conjunto de chaves.
# :param x: O dicionário de entrada.
# :param chaves: Chaves para navegar no dicionário.
# :return: O valor extraído ou None se a navegação falhar.
# """
    if isinstance(x, dict):
        for chave in chaves:
            x = x.get(chave)
            if x is None:
                return None
        return x
    return None

Valor original do DESTINATARIO: eduardo.socca@svriglobal.com
Valor original do DESTINATARIO: None


In [3]:
#------------------------------------------SEÇÃO--------------------------------------
# TODO: API 
# Corpo do login a ser utilizado no acesso
body = {
    "nome": api_username,
    "password":api_password
}

# Obtençao do token de acesso à polotrial
auth_url = urljoin(api_url, "/sessions")

response = requests.post(auth_url, json = body)

# # Verificar a resposta
# print(f"Status Code: {response.status_code}")
# print(f"Headers: {response.headers}")
# print(f"Content: {response.text}")

# Extraindo o token
token = response.json()["token"]

# Incorporando a string Bearer para inserir
if token:
    auth_token = "Bearer " + token
else:
    print("Falha ao obter o token.")
    
headers = {"Authorization": auth_token}

In [4]:
# Endpoints

# TODO: Generica

rota_generica = api_url+"/generica?nested=true"
df_generica = requests.get(rota_generica, headers = headers).json()
df_generica = pd.DataFrame(df_generica)
df_generica_limpo=df_generica[['id', 'ds_descricao']]
df_generica_limpo.head()

#------------------------------------------ROTAS---------------------------------------------- 
#TODO: Acesso Protocolos
rota_protocolo = api_url+"/protocolo?nested=true"
df_protocolo = requests.get(rota_protocolo, headers = headers).json()
df_protocolo = pd.DataFrame(df_protocolo)

#TODO: Acesso Participantes
rota_participantes = api_url+"/participantes?nested=true"
df_participantes = requests.get(rota_participantes, headers = headers).json()
df_participantes = pd.DataFrame(df_participantes)

#TODO: Acesso Participantes_visita (Agenda)
rota_participante_visita = api_url+"/participante_visita?nested=true"
df_participante_visita = requests.get(rota_participante_visita, headers=headers).json()
df_participante_visita = pd.DataFrame(df_participante_visita)

#TODO: Acesso Participantes_visita_procedimentos
rota_visita_procedimentos = api_url+"/power_bi_participante_visita_procedimento"
df_visita_procedimentos = requests.get(rota_visita_procedimentos, headers = headers).json()
df_visita_procedimentos = pd.DataFrame(df_visita_procedimentos)

#TODO: Acesso Eventos Adversos
rota_evento_adverso = api_url+"/evento_adverso?nested=true"
df_evento_adverso = requests.get(rota_evento_adverso, headers = headers).json()
df_evento_adverso = pd.DataFrame(df_evento_adverso)  

# TODO: Flowchart
rota_flowchart = api_url +"/protocolo_flowchart"
df_flowchart = requests.get(rota_flowchart, headers = headers).json()
df_flowchart = pd.DataFrame(df_flowchart)

# TODO: Pessoas
rota_pessoas = api_url + "/pessoas?nested=true"
df_pessoas = requests.get(rota_pessoas, headers = headers).json()
df_pessoas = pd.DataFrame(df_pessoas)

# TODO: Protocolo Financeiro
rota_protocolo_financeiro = api_url+"/protocolo_financeiro?nested=true"
df_protocolo_financeiro = requests.get(rota_protocolo_financeiro, headers = headers).json()
df_protocolo_financeiro = pd.DataFrame(df_protocolo_financeiro)

# TODO: Recebimentos
rota_recebimento = api_url + "/recebimento?nested=true"
df_recebimento = requests.get(rota_recebimento, headers = headers).json()
df_recebimento = pd.DataFrame(df_recebimento)

# TODO: Pagamentos
rota_pagamentos = api_url + "/pagamento?nested=true"
df_pagamento = requests.get(rota_pagamentos, headers = headers).json()
df_pagamento = pd.DataFrame(df_pagamento)

In [5]:

dim_visita_procedimentos = df_visita_procedimentos.copy()

# Selecionando os dados
procedimentos_extras=dim_visita_procedimentos[[
    'dados_protocolo_procedimento.co_protocolo',
    'dados_participante_visita.co_participante',
    'dados_participante_visita.nome_tarefa',
    'data_executada',
    'opcional',
    'dados_protocolo_procedimento.nome_procedimento_estudo',
    'dados_participante_visita_procedimento_executor.dados_pessoa_executor.ds_nome',
    'dados_visita_procedimento.opcional',
    'dados_visita_procedimento.disponibilidade',
    'co_visita_procedimento',
    'dados_nota_fiscal.codigo_nota_fiscal'
]]

In [6]:
# procedimentos_extras=dim_visita_procedimentos.copy()

# Renomeando as colunas
procedimentos_extras.rename(columns={
    'dados_protocolo_procedimento.co_protocolo': 'co_protocolo',
    'dados_participante_visita.co_participante':'co_participante',
    'dados_participante_visita.nome_tarefa': 'Visita',
    'data_executada':'Data Executada',
    'opcional': 'Tipo de procedimento' ,
    'dados_protocolo_procedimento.nome_procedimento_estudo': 'Procedimento',
    'dados_participante_visita_procedimento_executor.dados_pessoa_executor.ds_nome':'Executor',
    'dados_visita_procedimento.opcional': 'Opcional 2',
    'dados_visita_procedimento.disponibilidade': 'Disponibilidade',
    'co_visita_procedimento': 'Codigo Procedimentos',
    'dados_nota_fiscal.codigo_nota_fiscal':'codigo_nota_fiscal'
    }, inplace=True)

C:\Users\Eduardo Socca\AppData\Local\Temp\ipykernel_29144\1552288744.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procedimentos_extras.rename(columns={


In [7]:
# Merge com a tabela protocolo para obter os dados dos estudos
protocolo = df_protocolo[[
    'id',
    'apelido_protocolo',
    'dados_co_centro',
    'dados_tipo_de_iniciativa'
    ]].copy()


extrair = ['dados_co_centro','dados_tipo_de_iniciativa']
for coluna in extrair:
    protocolo[coluna] = protocolo[coluna].apply(extrair_ultima_informacao)
    
protocolo.rename(columns={
    'id':'co_protocolo',
    'apelido_protocolo': 'Protocolo',
    'dados_co_centro':'Centro',
    'dados_tipo_de_iniciativa': 'Iniciativa'
    }, inplace=True)
procedimentos_extras = procedimentos_extras.merge(protocolo, how = 'left', on='co_protocolo')

In [8]:
# Merge com a tabela Participantes para obter os dados do participante
participantes = df_participantes[[
    'id',
    'id_participante',
    'dados_status',
    'Braco'
    ]].copy()

extrair2 = ['dados_status','Braco']
for coluna in extrair2:
    participantes[coluna] = participantes[coluna].apply(extrair_ultima_informacao)
    
participantes.rename(columns={
    'id':'co_participante',
    'id_participante': 'Participante',
    'dados_status': 'Status do participante',
    'Braco': 'Braço no estudo'
    }, inplace=True)

procedimentos_extras = procedimentos_extras.merge(participantes, how = 'left', on='co_participante')

In [9]:
# Reordenando a nova tabela para melhor exibição
procedimentos_extras=procedimentos_extras[[
    'Protocolo','Iniciativa', 'Centro', 'Participante', 'Status do participante', 'Visita','Braço no estudo','Data Executada', 'Procedimento', 'Executor'
]]

#Categorizando a coluna data
colunas_data = ['Data Executada']

# Converte cada coluna de data separadamente para melhorar o desempenho
for coluna in colunas_data:
    procedimentos_extras[coluna] = pd.to_datetime(procedimentos_extras[coluna], errors='coerce').dt.tz_localize(None)

In [10]:
# # Filtrando os estudos patrocinados
# filtro3 = ['Patrocinador']
# procedimentos_extras=procedimentos_extras[procedimentos_extras['Iniciativa'].isin(filtro3)]

In [11]:
# Exibindo somente os dados duplicados quanto ao protocolo, centro, participante, visita, data do procedimento e procedimento.
procedimentos_extras = procedimentos_extras[procedimentos_extras.duplicated(subset =['Protocolo', 'Centro', 'Participante','Visita','Data Executada','Procedimento'], keep=False)]


In [ ]:
# # Filtrar o período de interesse
procedimentos_duplicados = procedimentos_extras[
    (procedimentos_extras['Data Executada']>= ultima_semana) #Após a primeira leva, alterar para delay de 1 semana
]

In [ ]:
# Primeira período para titulo do email
if not procedimentos_duplicados.empty:
    procedimentos_duplicados_min = procedimentos_duplicados['Data Executada'].min().strftime('%d/%m/%Y')
    procedimentos_duplicados_max = procedimentos_duplicados['Data Executada'].max().strftime('%d/%m/%Y')
else:
    procedimentos_duplicados_min = None
    procedimentos_duplicados_max = None

In [ ]:
# Função para criar a tabela do corpo do email 
def filtrar_protocolo_procedimentos_duplicados(dataframe, anos=3):
    # Filtrar contratos com data de assinatura não nula
    dataframe = dataframe.loc[dataframe['Data Executada'].notna(), :]

# Verificar se o DataFrame filtrado está vazio
    if procedimentos_duplicados.empty:
        return "Sem procedimentos Condicionais e/ou Extras"
    else:
        # Formatar o DataFrame para exibição em HTML
        dataframe_filtrado = procedimentos_duplicados.style\
            .format(precision=3, thousands=".", decimal=',')\
            .format_index(str.upper, axis=1)\
            .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
            .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

        return dataframe_filtrado.to_html(index=False)

# Chamando a função
procedimentos_duplicados_html = filtrar_protocolo_procedimentos_duplicados(procedimentos_duplicados)